In [5]:
import numpy as np
norm = np.linalg.norm
dot = np.dot
vector = np.ndarray
import sympy as sp

In [3]:
!pip install sympy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf356191/sympy-1.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for mpmath<1.4,>=1.1.0 from https://files.pythonhosted.org/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae62209/mpmath-1.3.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
    --------------------------------------- 0.1/6.3 MB 2.4 MB/s eta 0:00:03
   --------- ------------------------------ 1.5/6.3 MB 15.4 MB/s eta 0:00:01
   ---------------------- ----------------- 3.6/6.3 MB 25.4 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.3 MB 33.5 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 26.9 MB/s eta 0:0


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
class projector():

    def __init__(self, p1, p2, delta_0, epsilon, x_0, z_0, dim, f):
        self.p1: float = p1
        self.p2: float = p2
        self.delta_0: float = delta_0
        self.epsilon: float = epsilon
        self.epsilon1 = 1e-6
        self.epsilon2 = 1e-2
        self.x_0: vector = x_0
        self.z_0: vector = z_0
        self.f: callable = f
        self.dim : int = dim
        self.iteration_number: int = 0
    

    def gradient(self, x, h=1e-6):
        """
        Compute the gradient of f at point x.
        
        Parameters
        ----------
        f : callable
            Function f(x) -> scalar, where x is a numpy array
        x : numpy array
            Point at which to evaluate gradient
        h : float, optional
            Step size for finite differences
        
        Returns
        -------
        grad : numpy array
            Gradient vector of f at x
        """
        x = np.array(x, dtype=float)
        grad = np.zeros_like(x)
        
        for i in range(len(x)):
            x_forward = x.copy()
            x_backward = x.copy()
            x_forward[i] += h
            x_backward[i] -= h
            grad[i] = (self.f(x_forward) - self.f(x_backward)) / (2*h)
        
        return grad
    

    def hessian(self, x, h=1e-5):
        """
        Compute numerical Hessian of f at point x.
        
        Parameters
        ----------
        f : callable
            Function f(x) -> scalar, where x is a numpy array
        x : numpy array
            Point at which to evaluate Hessian
        h : float
            Step size for finite differences
        
        Returns
        -------
        H : numpy array (n x n)
            Hessian matrix
        """
        f = self.f
        x = np.array(x, dtype=float)
        n = len(x)
        H = np.zeros((n, n))
        
        for i in range(n):
            for j in range(n):
                # perturb in i and j directions
                x_pp = x.copy(); x_pp[i] += h; x_pp[j] += h
                x_pm = x.copy(); x_pm[i] += h; x_pm[j] -= h
                x_mp = x.copy(); x_mp[i] -= h; x_mp[j] += h
                x_mm = x.copy(); x_mm[i] -= h; x_mm[j] -= h
                
                H[i, j] = (f(x_pp) - f(x_pm) - f(x_mp) + f(x_mm)) / (4*h*h)
        return H



    def psi(self, x):
        gradx = self.gradient(x)
        grad_norm = norm(gradx)
        norm_cube = norm(x)**3
        return -1 /(norm_cube) * x + (dot(x, gradx)/(norm_cube * grad_norm**2)) * gradx
        pass

    def chi(self,x,z):
        gradx = self.gradient(x)
        hessx = self.hessian(x)
        return ((z.T @ hessx.T @ z)/(norm(gradx)**2)) * (gradx)
        pass

    def solve(self):
        if self.iteration_number==0:
            x = self.x_0
            z = np.zeros(self.dim, dtype = float)
            delta = self.delta_0
        

        while(norm(self.psi(x)) > self.epsilon):
            #step1: find x_hat
            x_hat = x + delta * z
            grad_xhat = self.gradient(x_hat)
            x_new = x_hat - (grad_xhat)/(norm(grad_xhat)**2) * (self.f(x_hat))
            some_complicated_expression = norm(x_new - x_hat)/ norm(x_hat)
            while (self.iteration_number!=0) and (some_complicated_expression < self.epsilon1 or some_complicated_expression > self.epsilon2):
                if some_complicated_expression < self.epsilon1:
                    delta *= 2
                else:
                    delta /= 2
                x_hat = x + delta * z
                grad_xhat = self.gradient(x_hat)
                x_new = x_hat - (grad_xhat)/(norm(grad_xhat)**2) * (self.f(x_hat))
                some_complicated_expression = norm(x_new - x_hat)/ norm(x_hat)
            
            # step 3: 
            z_new = z + delta * (self.p1 * self.psi(x) - self.p2 * z - self.chi(x,z))

            x = x_new
            z = z_new
            
            self.iteration_number+=1
            print(self.iteration_number, x, z)
        
        return x,z




In [34]:
def ellipsoid(x):
    A = np.array([[1,0,0],
                  [0,5,-2],
                  [0,-2,5]])
    c = np.array([11,-9,13])

    return  (x-c).T @ A.T @ (x-c) - 1.0

x = np.array([11,-9,13])
ellipsoid(x_intersect)

np.float64(6.661338147750939e-16)

In [33]:
# finding x_0 and z_0
# import numpy as np

def ellipsoid_line_intersection(A, c):
    """
    Find the intersection of the line from origin to center with ellipsoid boundary.
    
    Ellipsoid: (x - c)^T A (x - c) = 1
    Line: x = t * c
    
    Parameters
    ----------
    A : np.ndarray
        Positive definite matrix defining ellipsoid
    c : np.ndarray
        Center of the ellipsoid
    
    Returns
    -------
    x_intersect : np.ndarray
        Intersection point on the ellipsoid along line from origin to center
    """
    c = np.array(c, dtype=float)
    A = np.array(A, dtype=float)
    
    cAc = c.T @ A @ c
    t1 = 1 + 1/np.sqrt(cAc)
    t2 = 1 - 1/np.sqrt(cAc)
    
    # choose t that lies between 0 and 1 (line from origin to center)
    t = t2 if 0 <= t2 <= 1 else t1
    
    x_intersect = t * c
    return x_intersect

A = np.array([[1,0,0],
                  [0,5,-2],
                  [0,-2,5]])
c = np.array([11,-9,13])
x_intersect = ellipsoid_line_intersection(A,c)
x_intersect


array([10.74349146, -8.79012938, 12.69685355])

In [40]:
z_0 = np.zeros(3, dtype=float)
z_0[0]=1
pro = projector(
    p1=100,
    p2=2,
    delta_0=1,
    epsilon=1e-6,
    dim=3,
    x_0=x_intersect,
    z_0=np.array([0,0,0]),
    f=ellipsoid
)

ans = pro.solve()
ans



1 [10.74349146 -8.79012938 12.69685355] [-0.13674581 -0.0269153  -0.004901  ]
2 [10.60744839 -8.81978031 12.69535878] [ 0.00085779 -0.00553667  0.00647244]
3 [10.60689257 -8.82001269 12.69518997] [-0.12058347 -0.01933281 -0.00119388]
4 [10.48891455 -8.84652382 12.70338605] [ 0.00196535 -0.00946767  0.01153221]
5 [10.48739492 -8.84715991 12.7031897 ] [-0.10322914 -0.01272596  0.00137899]
6 [10.38892572 -8.86986767 12.71815195] [ 0.00317824 -0.01185234  0.01508123]
7 [10.38628786 -8.87095711 12.71820829] [-0.08486194 -0.00693321  0.00268879]
8 [10.30822114 -8.8892591  12.73684721] [ 0.00441134 -0.01283589  0.01712429]
9 [10.30464802 -8.89069295 12.73740765] [-0.06624255 -0.00183961  0.00246433]
10 [10.24678545 -8.90416948 12.75654817] [ 0.00563106 -0.01268395  0.01779887]
11 [10.24279088 -8.90568972 12.75767875] [-0.04879215  0.00247813  0.00063605]
12 [10.20331703 -8.91439484 12.77452357] [ 0.00681869 -0.01180614  0.01741131]
13 [10.19955928 -8.91568799 12.77602833] [-0.03421253  0.0057

(array([10.13999322, -8.92338998, 12.81344169]),
 array([ 0.00298485, -0.0025638 ,  0.00370092]))

In [48]:
x,z = ans

def check_feasibility(x, A, c, tol=1e-4):
    val = (x - c).T @ A @ (x - c) - 1.0
    return abs(val) <= tol, val

# Example:
ok, residual = check_feasibility(np.ravel(x), A, c)
# ok
# residual
ellipsoid(x)

np.float64(0.0001460862168110033)